In [1]:
from pathlib import Path

from bertviz import head_view

import torch

In [2]:
from model import model_load 
from test_model import load_config, run_model_for_attention

In [3]:
main_path = Path('.')
data_path = main_path / 'resource'
chpt_path = main_path / 'runs_copy' / 'best_SWAN_P1'

In [4]:
tokenizer = torch.load(chpt_path / 'tokenizer.pt')
checkpoint = torch.load(chpt_path / 'SWANPhase1Only.pt')

In [5]:
#nvix = model_load(chpt_path)
config = load_config(chpt_path)

In [6]:
config

{'seed': 1,
 'batch_size': 16,
 'beam_for_equation': 3,
 'beam_for_explanation': 5,
 'dataset': '/home/bydelta/SimpleFESTA/resource/dataset/pen.json',
 'learner': {'model': 'SWAN_P1',
  'encoder': 'google/electra-base-discriminator',
  'equation': {'hidden_dim': 0, 'intermediate_dim': 0, 'layer': 6, 'head': 0},
  'explanation': {'encoder': 'google/electra-base-discriminator',
   'shuffle': True}},
 'resource': {'num_gpus': 1.0, 'num_cpus': 1.0},
 'experiment': {'dev': {'split_file': '/home/bydelta/SimpleFESTA/resource/experiments/pen/dev',
   'period': 100},
  'train': {'split_file': '/home/bydelta/SimpleFESTA/resource/experiments/pen/train'},
  'test': {'split_file': '/home/bydelta/SimpleFESTA/resource/experiments/pen/test',
   'period': 500}},
 'grad_clip': 10.0,
 'optimizer': {'type': 'lamb',
  'lr': 0.00176,
  'betas': (0.9, 0.999),
  'eps': 1e-08,
  'debias': True},
 'scheduler': {'type': 'warmup-linear',
  'num_warmup_epochs': 10.0,
  'num_total_epochs': 500},
 'window': 3}

In [7]:
dataset_file = data_path / 'dataset' / 'pen.json'

KEY_DATASET = 'dataset'
KEY_SEED = 'seed'

exp_base = {
        KEY_DATASET: str(dataset_file.absolute()),
        KEY_SEED: config['seed']
    }

torch.backends.cudnn.deterministic = False

In [8]:
test_result = run_model_for_attention(chpt_path, **exp_base)

[W Context.cpp:70] Warning: torch.use_deterministic_algorithms is in beta, and its design and functionality may change in the future. (function operator())
Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at google/electra-base-discriminator were

Training on cuda

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RuntimeError: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility